In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
from sklearn import metrics

In [10]:
sonar = pd.read_csv("data/sonar.csv",header=None, na_values='?').dropna()
sonar.head()
print('\nDataFrame datatypes :\n', sonar.dtypes)


DataFrame datatypes :
 0     float64
1     float64
2     float64
3     float64
4     float64
       ...   
56    float64
57    float64
58    float64
59    float64
60     object
Length: 61, dtype: object


In [11]:
sonar.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [12]:
Xson = sonar.drop([60], axis=1) # X variable is the dataset without the class column
Xson = Xson.values 
Yson = sonar.iloc[:,60].values
Yson_int = np.empty((208))
for i in range(np.shape(Yson)[0]):
    if Yson[i] == 'R':
        Yson_int[i] = 0
    if Yson[i] == 'M':
        Yson_int[i] = 1

Standardizing our data

In [14]:
X_mean_son = np.reshape(Xson.mean(0), (-1,60))
X_std_son = np.reshape(Xson.std(0), (-1,60))
X_stand_son = (Xson - X_mean_son) / X_std_son  

Separate into training and validation

In [17]:
import sklearn.model_selection as cv
XTrain_son, XTest_son, YTrain_son, YTest_son = cv.train_test_split(X_stand_son, Yson_int,
    test_size=0.2, random_state=1)

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

We can do a small grid search to find the best solving method for LDA.

In [22]:
lda = LinearDiscriminantAnalysis()

grid = dict()
grid['solver'] = ['svd', 'lsqr', 'eigen']

cv_lda = GridSearchCV(estimator=lda, param_grid=grid, scoring='accuracy', cv=ms.KFold(n_splits=10), n_jobs=-1)
results_lda = cv_lda.fit(XTrain_son, YTrain_son)

In [28]:
print(results_lda)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearDiscriminantAnalysis(), n_jobs=-1,
             param_grid={'solver': ['svd', 'lsqr', 'eigen']},
             scoring='accuracy')


SVD is the best solving method.

Now we can tune the number of neighbors in the KNN classifier.

In [29]:
from sklearn import neighbors
neighbor= np.arange(1,16)
param_grid=dict(n_neighbors=neighbor)

knn = neighbors.KNeighborsClassifier()
grid_search=GridSearchCV(knn, param_grid=param_grid, cv=ms.KFold(n_splits=13), scoring='accuracy')
grid_search.fit(XTrain_son, YTrain_son) 

GridSearchCV(cv=KFold(n_splits=13, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])},
             scoring='accuracy')

The best number of neighbors is 4.

Now we have some functions I made to return some key performance metrics for each of the classifiers.

In [30]:
def classification_metrics(Y_true, Y_pred):
    n = np.shape(Y_true)[0]
    class metric:
        matrix = np.empty((2,2))
        recall = float()
        specificity = float()
        fallout = float()
        ppv = float()
        accuracy = float()
        missclassification = float()
    metrics = metric()
    tp = tn = fp = fn = 0
    for i in range(n):
        if Y_true[i] == Y_pred[i]:
            if Y_true[i] == 1:
                tp += 1
            if Y_true[i] == 0:
                tn += 1
        if Y_true[i] != Y_pred[i]:
            if Y_true[i] == 1:
                fn += 1
            if Y_true[i] == 0:
                fp += 1
    confusion = np.array([[tp, fn], [fp, tn]])
    metrics.matrix = confusion
    metrics.recall = confusion[0,0]/(confusion[0,0]+confusion[0,1])
    metrics.specificity = confusion[1,1]/(confusion[1,1]+confusion[1,0])
    metrics.fallout = confusion[1,0]/(confusion[1,0]+confusion[1,1])
    metrics.ppv = confusion[0,0]/(confusion[0,0]+confusion[1,0])
    metrics.accuracy = (confusion[0,0]+confusion[1,1])/np.sum(confusion)
    metrics.missclassification = (confusion[0,1]+confusion[1,0])/np.sum(confusion)
    
    return metrics

def print_metrics(metrics):
    print('The confusion matrix is        ', metrics.matrix[0,:])
    print('                               ', metrics.matrix[1,:])
    print('\n')
    print('The metrics are:        Fallout =', round(metrics.fallout,4),'               Recall =', round(metrics.recall,4))
    print('                    Specificity =', round(metrics.specificity,4), '                  PPV =', round(metrics.ppv,4))
    print('                       Accuracy =', round(metrics.accuracy,4), ' Missclass Error Rate =', round(metrics.missclassification,4))
     

First the LDA metrics

In [31]:
lda=LinearDiscriminantAnalysis(solver='svd') # solver found from GridSearchCV
lda.fit(XTrain_son, YTrain_son)
lda_pred_train=lda.predict(XTrain_son)
lda_pred_test=lda.predict(XTest_son)
lda_train_metrics = classification_metrics(YTrain_son, lda_pred_train)
lda_test_metrics = classification_metrics(YTest_son, lda_pred_test)
print('LDA Training Metrics:')
print_metrics(lda_train_metrics)
print('\n')
print('LDA Testing Metrics:')
print_metrics(lda_test_metrics)

LDA Training Metrics:
The confusion matrix is         [84  7]
                                [ 8 67]


The metrics are:        Fallout = 0.1067                Recall = 0.9231
                    Specificity = 0.8933                   PPV = 0.913
                       Accuracy = 0.9096  Missclass Error Rate = 0.0904


LDA Testing Metrics:
The confusion matrix is         [15  5]
                                [ 8 14]


The metrics are:        Fallout = 0.3636                Recall = 0.75
                    Specificity = 0.6364                   PPV = 0.6522
                       Accuracy = 0.6905  Missclass Error Rate = 0.3095


now the QDA

In [32]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(XTrain_son, YTrain_son)
qda_pred_train=qda.predict(XTrain_son)
qda_pred_test=qda.predict(XTest_son)
qda_train_metrics = classification_metrics(YTrain_son, qda_pred_train)
qda_test_metrics = classification_metrics(YTest_son, qda_pred_test)
print('QDA Training Metrics:')
print_metrics(qda_train_metrics)
print('\n')
print('QDA Testing Metrics:')
print_metrics(qda_test_metrics)

QDA Training Metrics:
The confusion matrix is         [91  0]
                                [ 0 75]


The metrics are:        Fallout = 0.0                Recall = 1.0
                    Specificity = 1.0                   PPV = 1.0
                       Accuracy = 1.0  Missclass Error Rate = 0.0


QDA Testing Metrics:
The confusion matrix is         [20  0]
                                [12 10]


The metrics are:        Fallout = 0.5455                Recall = 1.0
                    Specificity = 0.4545                   PPV = 0.625
                       Accuracy = 0.7143  Missclass Error Rate = 0.2857


We can make our logistic model now.

In [33]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression()
logit.fit(XTrain_son, YTrain_son)
logit_pred_train = logit.predict(XTrain_son)
logit_pred_test = logit.predict(XTest_son)
logit_train_metrics = classification_metrics(YTrain_son, logit_pred_train)
logit_test_metrics = classification_metrics(YTest_son, logit_pred_test)

In [34]:
print('Logit Training Metrics:')
print_metrics(logit_train_metrics)
print('\n')
print('Logit Testing Metrics:')
print_metrics(logit_test_metrics)

Logit Training Metrics:
The confusion matrix is         [86  5]
                                [ 4 71]


The metrics are:        Fallout = 0.0533                Recall = 0.9451
                    Specificity = 0.9467                   PPV = 0.9556
                       Accuracy = 0.9458  Missclass Error Rate = 0.0542


Logit Testing Metrics:
The confusion matrix is         [15  5]
                                [ 7 15]


The metrics are:        Fallout = 0.3182                Recall = 0.75
                    Specificity = 0.6818                   PPV = 0.6818
                       Accuracy = 0.7143  Missclass Error Rate = 0.2857


And lastly we make our KNN classifier

In [35]:
knn = neighbors.KNeighborsClassifier(n_neighbors=4)# Found from GridSearchCV
knn.fit(XTrain_son, YTrain_son)
knn_pred_train = knn.predict(XTrain_son)
knn_pred_test = knn.predict(XTest_son)
knn_train_metrics = classification_metrics(YTrain_son, knn_pred_train)
knn_test_metrics = classification_metrics(YTest_son, knn_pred_test)

In [36]:
print('KNN Training Metrics:')
print_metrics(knn_train_metrics)
print('\n')
print('KNN Testing Metrics:')
print_metrics(knn_test_metrics)

KNN Training Metrics:
The confusion matrix is         [81 10]
                                [ 5 70]


The metrics are:        Fallout = 0.0667                Recall = 0.8901
                    Specificity = 0.9333                   PPV = 0.9419
                       Accuracy = 0.9096  Missclass Error Rate = 0.0904


KNN Testing Metrics:
The confusion matrix is         [17  3]
                                [ 5 17]


The metrics are:        Fallout = 0.2273                Recall = 0.85
                    Specificity = 0.7727                   PPV = 0.7727
                       Accuracy = 0.8095  Missclass Error Rate = 0.1905


In [37]:
print('LDA missclassification error rate for training is'\
      , round(lda_train_metrics.missclassification,4), ' and for testing is'\
      , round(lda_test_metrics.missclassification,4))
print('QDA missclassification error rate for training is'\
      , round(qda_train_metrics.missclassification,4), ' and for testing is'\
      , round(qda_test_metrics.missclassification,4))
print('Logit missclassification error rate for training is'\
      , round(logit_train_metrics.missclassification,4), ' and for testing is'\
      , round(logit_test_metrics.missclassification,4))
print('KNN missclassification error rate for training is'\
      , round(knn_train_metrics.missclassification,4), ' and for testing is'\
      , round(knn_test_metrics.missclassification,4))

LDA missclassification error rate for training is 0.0904  and for testing is 0.3095
QDA missclassification error rate for training is 0.0  and for testing is 0.2857
Logit missclassification error rate for training is 0.0542  and for testing is 0.2857
KNN missclassification error rate for training is 0.0904  and for testing is 0.1905


We can see that QDA and Logit models both have the same testing misclassification error rates, and very similar training error rates. To decide on which of these two models to use, we need to break up the misclassification error rates into specific types of misclassification. For the scenario that this dataset is from, a false positive is not detrimental as you are classifying a rock as a mine and which leads to more caution that actually required. The false negatives, however, are much more dangerous as in that situation you would be classifying a dangerous mine as a rock, which could be disastrous. So let us look at the rate of false negatives in testing.

In [38]:
qda_false_neg = qda_test_metrics.matrix[0,1]/(qda_test_metrics.matrix[0,1] + qda_test_metrics.matrix[0,0])
logit_false_neg = logit_test_metrics.matrix[0,1]/(logit_test_metrics.matrix[0,1] + logit_test_metrics.matrix[0,0])
print('QDA False Negative Rate is', round(qda_false_neg,4))
print('Logit False Negative Rate is', round(logit_false_neg,4))

QDA False Negative Rate is 0.0
Logit False Negative Rate is 0.25


The QDA classifier has a much lower false negative rate than the logit function. So we can conclude that the QDA classifier is the best for this data.